In [1]:
import flywheel
import flywheel_gear_toolkit as gt
from pprint import pprint

In [2]:
fw = flywheel.Client()

In [3]:
user_info = fw.get_current_user()
print(f"Firstname: {user_info.firstname} \n"
      f"Lastname: {user_info.lastname} \n"
      f"Email: {user_info.email}")

Firstname: Xuchu 
Lastname: Liu 
Email: Xuchu_Liu@rush.edu


In [4]:
project_sessions = fw.sessions()
for session in project_sessions:
    print(session.label)

2004-08-17 10_58_48
07-03-08 11_43 AM
07-12-04 12_34 PM
09-18-07 9_27 AM
09-01-05 9_18 AM
OHIF_6
OHIF_11
exam_000007


In [5]:
my_session_label = 'OHIF_6'
my_session_id = fw.sessions.find(f'label={my_session_label}')[0].id
print(my_session_id)

61f008e591188d92013e6c34


In [6]:
my_session = fw.get_session(my_session_id)
# print(my_session)

In [7]:
my_group = my_session.parents.group
print(my_group)

fetal


In [8]:
my_metadata = my_session.info
ohif_metadata = my_metadata['ohifViewer']

In [9]:
pprint(ohif_metadata['measurements']['Length'])

[]


In [10]:
Pons_Length = {}
Pons_Length['description'] = 'A-P Diameter of Pons'

In [11]:
Vermis_Length = {}
Vermis_Length['description'] = 'A-P Diameter of Vermis'

In [12]:
HVermis_Length = {}
HVermis_Length['description'] = 'Height of Vermis'

In [13]:
Fronto_Length = {}
Fronto_Length['description'] = 'Fronto-Occpitial'

In [14]:
ohif_metadata['measurements']['Length'].append(Pons_Length)
ohif_metadata['measurements']['Length'].append(Vermis_Length)
ohif_metadata['measurements']['Length'].append(HVermis_Length)
ohif_metadata['measurements']['Length'].append(Fronto_Length)
# ohif_metadata['measurements']['Length'] = [] # delete

### import_data.py

In [20]:
# If the "User Origin" column is not present in the Dataframe, generate it using
# the user ID of the person running this gear (or logged into the flywheel client)
user = fw.get_current_user()
user_id = user.id
print(user_id)

Xuchu_Liu@rush.edu


In [23]:
context = gt.GearToolkitContext()
context.config_from_args([])
destination = context.destination

In [34]:
destination = fw.get(my_session_id) # destination = my_session
group = fw.get_group(destination.parents.group) #  group object
project = fw.get_project(destination.parents.project) # project object

{'analyses': [{'created': datetime.datetime(2022, 1, 12, 3, 11, 13, 67000, tzinfo=tzutc()),
               'description': '',
               'files': [{'classification': {},
                          'created': datetime.datetime(2022, 1, 12, 3, 17, 4, 909000, tzinfo=tzutc()),
                          'deid_log_id': None,
                          'file_id': '61de4830dd5024fc5e3d3f36',
                          'hash': None,
                          'id': '2067ec2a-056a-4c78-b06e-e31cc073655b',
                          'info': {},
                          'info_exists': False,
                          'mimetype': 'application/octet-stream',
                          'modality': None,
                          'modified': datetime.datetime(2022, 1, 12, 3, 17, 4, 909000, tzinfo=tzutc()),
                          'name': 'info.csv',
                          'origin': {'id': '61de46d1ae5d9a7eaf3d435b',
                                     'method': None,
                             

In [43]:
group_name = group.id
print(group_name)
project_name = project.label
print(project_name)
my_subject_label = 'exam_000006'
subject = project.subjects.find(f'label={my_subject_label}')
# print(subject)

fetal
Matlab GUIs
[{'age': None,
 'analyses': [],
 'code': 'exam_000006',
 'cohort': None,
 'created': datetime.datetime(2022, 1, 25, 14, 27, 49, 537000, tzinfo=tzutc()),
 'ethnicity': None,
 'files': [],
 'firstname': None,
 'id': '61f008e591188d92013e6c33',
 'info': {},
 'info_exists': False,
 'label': 'exam_000006',
 'lastname': None,
 'master_code': None,
 'mlset': None,
 'modified': datetime.datetime(2022, 1, 25, 15, 8, 7, 47000, tzinfo=tzutc()),
 'notes': [],
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'fetal',
             'project': '61b7a25aece9fa9a563d3d26',
             'session': None,
             'subject': None},
 'permissions': [],
 'project': '61b7a25aece9fa9a563d3d26',
 'race': None,
 'revision': 3,
 'sex': None,
 'species': '',
 'strain': '',
 'tags': [],
 'type': None}]


In [54]:
############################################################################
# STEP 1: Loop through subject/session combos and aggregate file matches  #
############################################################################
# We will first loop through and find any and all matches for each subject/session combination.
# Group by subject/session combos, to minimize loading.
my_file_name = '7 - SAG BrainT2 HASTE BH.dicom.zip'
my_acquisitions = my_session.acquisitions()
my_acquisition = {}
my_file = None
for acquisition in my_acquisitions:
    for file in acquisition.files[:1]:
        if file.name.find(f'{my_file_name}') == 0:
            my_file = file # flywheel.FileEntry
            my_acquisition = acquisition
            break
my_acquisition_label = my_acquisition.label

In [53]:
############################################################################
# STEP 2: Loop through aggregated matches and ensure there is only one     #
# Match per row                                                            #
############################################################################

# Now that we have assembled all the possible matches for every index in the dataframe, go through and make sure
# There aren't duplicates.
path = f"{group_name}/{project_name}/{my_subject_label}/{my_session_label}/{my_acquisition_label}/{my_file_name}"

In [ ]:
# roi = cu.get_roi_from_row(series, match.file, ses)
# series = df.loc[index]
# Class Match:
#     file: flywheel.FileEntry = None
# parent = match.file.parent

### csv_utils.py

In [ ]:
# def get_roi_from_row(series, file, session):
#     """Generate the dictionaries from a pandas series to create the ROI in flywheel

#     Args:
#         series (pandas.Series): a row from a dataframe describing an ROI
#         file (flywheel.FileEntry): a flywheel file to attach the ROI to
#         session (flywheel.Session): the parent session of the file

#     Returns:
#         roi (ROI_Template.ROI): a custom ROI object

#     """
    # # Get the handle from the row.  The handle contains the x/y bounds of the ROI
    # handle = get_handle_from_row(series)
    # stats = get_stats_from_row(series)

In [ ]:
# def get_handle_from_row(series):
#     """Generate the flywheel handle from the series.

#     The only truly "required" columns needed to create a handle from the series are:
#     X min
#     Y min
#     X max
#     Y max

#     Args:
#         series (pandas.Series): a row from a dataframe describing an ROI

#     Returns:
#         handle_dict (dict): a nested dictionary object mirroring the "handle" object
#             used for the flywheel ROI metadata

#     """

In [15]:
# my_session.update_info(my_metadata)

1 - Scout Expiration BH.dicom.zip
10 - COR Body TRUEFIS BH.dicom.zip
11 - AXIAL BRAIN TRUEFIS BH4MM.dicom.zip
12 - AXIAL BRAIN TRUEFIS BH4MM.dicom
13 - AXIAL BRAIN TRUEFIS BH4MM.dicom.zip
14 - Scout Expiration BH.dicom.zip
15 - AX BodyTRUEFIS  BH.dicom.zip
16 - COR BodyTRUEFIS BH.dicom.zip
17 - COR Body TRUEFIS BH.dicom.zip
18 - COR BodyTRUEFIS BH.dicom.zip
2 - SAG BrainT2 HASTE BH.dicom.zip
3 - AX BrainT2 HASTE BH.dicom.zip
4 - COR BrainT2 HASTE BH.dicom.zip
5 - Scout Expiration BH.dicom.zip
6 - Sag Spine T2 HASTE BH.dicom.zip
7 - SAG BrainT2 HASTE BH.dicom.zip
8 - Sag Spine T2 HASTE BH.dicom.zip
9 - COR Spine T2 HASTE BH.dicom.zip
